<a href="https://colab.research.google.com/github/adanog/DataScienceImagingHub/blob/main/241016_LoadIamges_CreateMergedImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About the notebook
The purpose of this Jupyter Notebook is to open two images from two channels, one of brightfield and the other of fluorescence


# 00 - Special Instructions for Google Colab Users

The following lines of code should be executed only when running your script on Google Colab. This is crucial to leverage the additional features provided by Colab, most notably, the availability of a free GPU.  **If, you're running the code locally, this line can be skipped (GO TO STEP 01 - Loading dependencies) as it pertains specifically to the Colab setup.**

## Give access to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Create stacks from two channels, run only if necessary

The model require one single image from 2 channels in tif format. This cell is create single images from to brightfield and fluorescence image to a single one.

In [ ]:
!pip install Pillow numpy tifffile

In [ ]:
import os
import numpy as np
import tifffile as tiff

# Paths to the directories
campo_claro_dir = '/content/drive/MyDrive/.../Brightfield'
fluorescencia_dir = '/content/drive/MyDrive/.../Fluorescence'

# Get all TIFF files from both directories
campo_claro_files = sorted([os.path.join(campo_claro_dir, f) for f in os.listdir(campo_claro_dir) if f.endswith('.ome.tif')])
fluorescencia_files = sorted([os.path.join(fluorescencia_dir, f) for f in os.listdir(fluorescencia_dir) if f.endswith('.ome.tif')])

# Check that we have the same number of files in both directories
if len(campo_claro_files) != len(fluorescencia_files):
    raise ValueError("Number of images in 'Brightfield' and 'Fluorescence' must match.")

# Process each pair of files
for i, (file1_path, file2_path) in enumerate(zip(campo_claro_files, fluorescencia_files)):
    # Read the images using tifffile
    image1 = tiff.imread(file1_path)
    image2 = tiff.imread(file2_path)

    # Ensure both images have the same size
    if image1.shape != image2.shape:
        raise ValueError(f"Images {file1_path} and {file2_path} must be the same size to concatenate them as channels.")

    # Convert to appropriate shape if necessary
    if image1.ndim == 3 and image1.shape[0] > 1:
        image1 = image1.transpose(1, 2, 0)  # Change from (C, H, W) to (H, W, C)
    if image2.ndim == 3 and image2.shape[0] > 1:
        image2 = image2.transpose(1, 2, 0)

    # Stack the images along a new channel dimension
    concatenated = np.stack((image1, image2), axis=-1)  # Shape will be (H, W, C, 2)

    # Convert to shape (C, H, W) for TIFF
    concatenated = np.moveaxis(concatenated, -1, 0)  # Now shape is (2, H, W)

    # Save the concatenated image
    output_path = f'/content/drive/MyDrive/.../MergedChannels/concatenated_image_{i+1}.tif'
    tiff.imwrite(output_path, concatenated)

    # print(f"Concatenated image saved at {output_path} with shape: {concatenated.shape}")
